GAN 실습

In [1]:
from tensorflow.keras.datasets import mnist
(train_images, _), (_,_) = mnist.load_data()

In [2]:
# -1 ~ 1 사이로 표준화
train_images = (train_images.astype('float32') - 127.5 ) / 127.5
print(train_images.dtype)

float32


In [4]:
# channel 구조 변경
import numpy as np
Train_images = np.expand_dims(train_images, axis=-1)
print(train_images.shape)

(60000, 28, 28)


In [18]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).batch(256)
print(train_dataset)

<_BatchDataset element_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name=None)>


In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [7]:
#생성자
generator_model = Sequential([
    layers.Input(shape=(100,)),
    layers.Dense(7*7*256, activation='elu'),
    layers.BatchNormalization(),

    layers.Reshape((7,7,256)),
    layers.Conv2DTranspose(128, 5, padding='same', activation='elu'), #7*7*28
    layers.BatchNormalization(),

    layers.Conv2DTranspose(64, 5, strides=(2,2), padding='same', activation='elu'), #14*14*64
    layers.BatchNormalization(),

    layers.Conv2DTranspose(1, 5, strides=(2,2), padding='same', activation='tanh'), #28*28*1
])
generator_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 12544)             1266944   
                                                                 
 batch_normalization_3 (Bat  (None, 12544)             50176     
 chNormalization)                                                
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 7, 7, 128)         819328    
 Transpose)                                                      
                                                                 
 batch_normalization_4 (Bat  (None, 7, 7, 128)         512       
 chNormalization)                                                
                                                        

In [9]:
discriminator_model = Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(64, (5,5), strides=(2,2), padding='same', activation='elu'),
    layers.Dropout(0.3),

    layers.Conv2D(128, (5,5), strides=(2,2), padding='same', activation='elu'),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(1)
])
discriminator_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 1)                 6273      
                                                                 
Total params: 212865 (831.50 KB)
Trainable params: 212

In [10]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output) # 1
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output) # 1
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output) # 0
    total_loss = real_loss + fake_loss
    return total_loss
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [12]:
@tf.function
def train_step(images):
    noise = tf.random.normal([256, 100]) # 이미지 생성을 위한 노이즈 입력

    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape: # 미분을 위한 도구, loss를 계산
        generated_image = generator_model(noise, training=True)
        fake_output = discriminator_model(generated_image, training=True)
        real_output = discriminator_model(images, training=True)

        gen_loss = generator_loss(fake_output)
        dis_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator_model.trainable_variables)
    gradients_of_discriminator = dis_tape.gradient(dis_loss, discriminator_model.trainable_variables)

    generator_optimizer.apply_gradients( zip(gradients_of_generator, generator_model.trainable_variables))
    discriminator_optimizer.apply_gradients( zip(gradients_of_discriminator, discriminator_model.trainable_variables))